In [5]:
import numpy as np
import re
import json
import random
import sys
sys.path.append("../../")
from yelp_sentiment_attention.train import *
from yelp_multiclass.data.yelp_dataset import load_word_indices

from yelp_multiclass.data.textProcess import createTextSent,tokenizeText,buildIndexToWordDict,createNumericSent

## Restore trained model from file

In [6]:
log.info('Reading Vocab')
word_index = load_word_indices()
index_word = buildIndexToWordDict(word_index)
log.info('Done')

2018-07-13 22:31:29,464 - Yelp_Review_Processor - INFO - Reading Vocab
2018-07-13 22:31:31,167 - Yelp_Review_Processor - INFO - Done


## Pre-process reviews

In [24]:
reviews=[
    '''Pretty amazing stuff going on here.

Let me start off by saying there's literally a tub, A TUB, of pickles on the self serve counter.  YUP.  So thanks for that, Pearl.

I had a mini deluxe mushroom cheeseburger and it was beautiful.  I didn't take a picture because I lack the self-control to wait a few more seconds.  I also inhaled the spries (sweet potato/fries).  They were cooked to perfection, extra crispy!

It's a tiny location, so that might interfere with dining in to enjoy your fresh meal.  I would recommend to try planning your visit around a less busy time of day.  Honestly, I may not even consider my own words because I have no idea if/when I'd be back.'''
    ,
    '''Ordered a Mexican Chicken shwarma. So salty! Like soooo salty! Thats it! I still cant wash the salt off my tongue. How is this place rated so High. I got tricked.

I’m like cringey right now.

I threw it.

God awful.

Ps: packaging was so good I was like yeah. This will be great
Took two bites And I threw it.

Also I hate wasting food so imagine me throwing it. It was so salty. I just couldnt eat anymore.

Hangry as hell. 
    ''',
    '''Qureshi Bistro – From the famous Kakori house led by Qureshi’s here comes their extended brand by the name Qureshi Indo Arabic Bistro. It sits in one of the most competitive lanes of Oshiwara and serves cuisines from Lucknowi, Mughlai, European to Middle Eastern. Ambience wise, they have two set ups for quiet meets, indoors suits well with quirky wall arts and well lit whereas for a chilled out feel with bustling sounds of things around you, outdoors will be perfect. Staff was well knowledgeable about dishes served upon ordering and courteous too.

Things I had

Mocktails - Pineapple mojito (Rating 3/5) and tamarind margarita (Rating 4/5) – Refreshing drinks to start with where tamarind had very strong yet appealing taste with spiciness, some hints of guava and delight to drink the mojito was slightly over powered by sugar/sweetness innit. I certainly recommend tamarind margarita for the next visit.

Rosali Kebab (Rating 4/5) – It may look like a pretty average dish though made as breast chicken in outer layer and minced chicken kebab with hints of cheese had enough spices punch in the marinade. Though it were bit dry than usual rest all was perfect.'''
]

rev_numeric=[]

for r in reviews:
    r_n=createNumericSent(r,word_index)
    temp=[1] #1 denotes start of sentence
    temp.extend(r_n)
    rn=temp
    rev_numeric.append(r_n)
    #print(createTextSent(r_n,index_word))

X_input = zero_pad(rev_numeric, SEQUENCE_LENGTH)

for r in X_input:
    print(createTextSent(r,index_word))



 pretty amazing stuff going on here let me start off by saying there's literally a tub, a tub, of pickles on :UNK: self serve counter yup so thanks for that, pearl i had a mini deluxe mushroom cheeseburger and it was beautiful i didn't take a picture because i lack :UNK: self-control to wait a few more seconds i also inhaled :UNK: :UNK: (sweet :UNK: they were cooked to perfection, extra crispy it's a tiny location, so that might interfere with dining in to enjoy your fresh meal i would recommend to try planning your visit around a less busy time of day honestly, i may not even consider my own words because i have no idea if/when i'd be back  :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :PAD: :P

In [25]:
saver = tf.train.Saver()

log.info('Restorng saved model')

with tf.Session() as sess:
    saver.restore(sess, MODEL_PATH)
    y_batch_test = np.repeat(0,len(X_input)) 
    seq_len_test = np.array([list(x).index(0) + 1 for x in X_input])
    results = sess.run([alphas,y_hat], feed_dict={batch_ph: X_input, target_ph: y_batch_test,
                                                seq_len_ph: seq_len_test, keep_prob_ph: 1.0})

2018-07-13 22:52:27,655 - Yelp_Review_Processor - INFO - Restorng saved model
INFO:tensorflow:Restoring parameters from G:\work\nlp\datasets\yelp\attention\model\
2018-07-13 22:52:27,727 - tensorflow - INFO - Restoring parameters from G:\work\nlp\datasets\yelp\attention\model\


## Read Vocab from disk

## Read results from model and generate file

In [30]:
alphas_values = results[0][0]
filename="visualization3.html"
with open(filename, "w") as html_file:
    for X,alpha_values,Y_hat,Y in zip(X_input,results[0],results[1],y_batch_test ) :
        print(Y)
        print(Y_hat)
        html_file.write("<br/><hr/>");
        sample = X
        words = list(map(index_word.get, sample))
        html_file.write('%s  %s\n' % (Y, Y_hat));
        if Y_hat > 0:
            div_color='#b8efc8'
        else:
            div_color = '#ff848a'
        html_file.write('<div style="background-color:%s">\n' % div_color);
        for word, alpha in zip(words, alphas_values / alphas_values.max()):
            if word == ":START:":
                continue
            elif word == ":PAD:":
                break
            html_file.write('<font style="background: rgba(255, 255, 0, %f)">%s</font>\n' % (alpha, word))
        html_file.write('</div>\n');
print('Working director is ' +  os.getcwd())

0
-0.91714036
0
1.0265259
0
-2.0139637
Working director is K:\python_projects\KerasTextClassifier\yelp_sentiment_attention\test
